In [28]:
pip install pandas transformers


In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb_The Good, the Bad.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(100, 2)


In [31]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.6019343733787537},
 {'label': '0', 'score': 0.7331438064575195},
 {'label': '2', 'score': 0.3429885506629944},
 {'label': '1', 'score': 0.6220027208328247},
 {'label': '2', 'score': 0.9407133460044861},
 {'label': '1', 'score': 0.4732936918735504},
 {'label': '2', 'score': 0.7630813717842102},
 {'label': '1', 'score': 0.6065638661384583},
 {'label': '2', 'score': 0.2663687765598297},
 {'label': '1', 'score': 0.7279374003410339},
 {'label': '1', 'score': 0.5016587376594543},
 {'label': '1', 'score': 0.24874693155288696},
 {'label': '3', 'score': 0.5781169533729553},
 {'label': '2', 'score': 0.8290045261383057},
 {'label': '0', 'score': 0.6559361219406128},
 {'label': '1', 'score': 0.47031980752944946},
 {'label': '1', 'score': 0.3203935921192169},
 {'label': '2', 'score': 0.8377623558044434},
 {'label': '1', 'score': 0.5772743225097656},
 {'label': '2', 'score': 0.8726518750190735},
 {'label': '1', 'score': 0.48768675327301025},
 {'label': '2', 'score': 0.6957

In [32]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,Name one single modern movie where the protago...,2,0.601934
1,Was that a real or fictional graveyard?,0,0.733144
2,I am a Metalhead but Morricone was a genius,2,0.342989
3,One of my favorite movies definitely in my top...,1,0.622003
4,Bellissimo...uso questo video Michele. .per di...,2,0.940713
...,...,...,...
95,كنا نخلص درسه بسرعه من أجل اتابع هاي الأفلام ا...,1,0.527375
96,Шедевр.,2,0.485693
97,Taek,1,0.320014
98,❤❤❤super,2,0.401939


In [33]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         0        sadness
2         2           love
3         1            joy
4         2           love
..      ...            ...
95        1            joy
96        2           love
97        1            joy
98        2           love
99        2           love

[100 rows x 2 columns]


<ipython-input-33-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [34]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_The Good, the Bad_with_sentiments.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,Name one single modern movie where the protago...,2,0.601934,love
1,Was that a real or fictional graveyard?,0,0.733144,sadness
2,I am a Metalhead but Morricone was a genius,2,0.342989,love
3,One of my favorite movies definitely in my top...,1,0.622003,joy
4,Bellissimo...uso questo video Michele. .per di...,2,0.940713,love
...,...,...,...,...
95,كنا نخلص درسه بسرعه من أجل اتابع هاي الأفلام ا...,1,0.527375,joy
96,Шедевр.,2,0.485693,love
97,Taek,1,0.320014,joy
98,❤❤❤super,2,0.401939,love
